# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
type(stocks)

pandas.core.frame.DataFrame

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
data.keys()

dict_keys(['symbol', 'companyName', 'primaryExchange', 'calculationPrice', 'open', 'openTime', 'openSource', 'close', 'closeTime', 'closeSource', 'high', 'highTime', 'highSource', 'low', 'lowTime', 'lowSource', 'latestPrice', 'latestSource', 'latestTime', 'latestUpdate', 'latestVolume', 'iexRealtimePrice', 'iexRealtimeSize', 'iexLastUpdated', 'delayedPrice', 'delayedPriceTime', 'oddLotDelayedPrice', 'oddLotDelayedPriceTime', 'extendedPrice', 'extendedChange', 'extendedChangePercent', 'extendedPriceTime', 'previousClose', 'previousVolume', 'change', 'changePercent', 'volume', 'iexMarketPercent', 'iexVolume', 'avgTotalVolume', 'iexBidPrice', 'iexBidSize', 'iexAskPrice', 'iexAskSize', 'iexOpen', 'iexOpenTime', 'iexClose', 'iexCloseTime', 'marketCap', 'peRatio', 'week52High', 'week52Low', 'ytdChange', 'lastTradeTime', 'isUSMarketOpen'])

In [6]:
price = data["latestPrice"]
market_cap = data["marketCap"]

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [9]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns=my_columns)

In [ ]:
final_dataframe.append(
pd.Series([
    symbol,
    price,
    market_cap,
    "N/A"
], index=my_columns),     
    ignore_index = True)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
final_dataframe = pd.DataFrame(columns=my_columns)

In [ ]:
for stock_ticker in stocks["Ticker"]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock_ticker}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series([
        stock_ticker,
        data["latestPrice"],
        data["marketCap"],
        "N/A"
    ], index=my_columns), ignore_index=True)

In [ ]:
final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
#https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
ticker_groups = list(chunks(stocks["Ticker"], 100))
ticker_symbols_group = []
final_dataframe = pd.DataFrame(columns=my_columns)
for i in range(0, len(ticker_groups)):
    ticker_symbols_group.append(",".join(ticker_groups[i]))

for ticker_symbols in ticker_symbols_group:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={ticker_symbols}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    print(batch_api_url)
    data_request = requests.get(batch_api_url)
    print(data_request.status_code)
    if(data_request.status_code == 200):
        data = data_request.json()
        for symbol in ticker_symbols.split(","):
            final_dataframe = final_dataframe.append(pd.Series([
                symbol,
                data[symbol]["quote"]["latestPrice"],
                data[symbol]["quote"]["marketCap"],
                "N/A"
            ],index=my_columns), 
                                                     ignore_index=True)


https://sandbox.iexapis.com/stable/stock/market/batch?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=quote&token=Tpk_059b97af715d417d9f49f50b51b1c448
200
https://sandbox.iexapis.com/stable/stock/market/batch?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,

In [12]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.85,37570502185,N/A
1,AAL,25.20,16172775451,N/A
2,AAP,188.16,12151948958,N/A
3,AAPL,121.04,2038647035555,N/A
4,ABBV,105.97,184135949334,N/A
...,...,...,...,...
500,YUM,113.33,32967027402,N/A
501,ZBH,159.85,34008284921,N/A
502,ZBRA,462.65,24998277403,N/A
503,ZION,59.40,9818101469,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [13]:
portfolio_value = None

while portfolio_value is None:
#     portfolio_value_string = input("Enter the value of your portfolio... ")
    portfolio_value_string = 10000000
    try:
        portfolio_value = float(portfolio_value_string)
        print(portfolio_value)
    except ValueError:
        print("That's not a number! Enter a numerical value of your portfolio in $.")

10000000.0


In [14]:
position_size = portfolio_value/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])


In [15]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124.85,37570502185,158
1,AAL,25.20,16172775451,785
2,AAP,188.16,12151948958,105
3,AAPL,121.04,2038647035555,163
4,ABBV,105.97,184135949334,186
...,...,...,...,...
500,YUM,113.33,32967027402,174
501,ZBH,159.85,34008284921,123
502,ZBRA,462.65,24998277403,42
503,ZION,59.40,9818101469,333


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [37]:
writer = pd.ExcelWriter("recommended_trades.xlsx", engine = "xlsxwriter")
final_dataframe.to_excel(writer, "recommended_trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [38]:
background_color = "#ffffff"
font_color = "#0a0a23"

string_format = writer.book.add_format(
{
    "font_color": font_color,
    "bg_color": background_color,
    "border": 1
})

dollar_format = writer.book.add_format(
{
    "num_format": "$0.00",
    "font_color": font_color,
    "bg_color": background_color,
    "border": 1
})

integer_format = writer.book.add_format(
{
    "num_format": 0,
    "font_color": font_color,
    "bg_color": background_color,
    "border": 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [39]:
# writer.sheets['recommended_trades'].set_column('A:A', #This tells the method to apply the format to column B
#                      18, #This tells the method to apply a column width of 18 pixels
#                      string_format #This applies the format 'string_template' to the column
#                     )

# writer.sheets['recommended_trades'].set_column('B:B', #This tells the method to apply the format to column B
#                      18, #This tells the method to apply a column width of 18 pixels
#                      string_format #This applies the format 'string_template' to the column
#                     )

# writer.sheets['recommended_trades'].set_column('C:C', #This tells the method to apply the format to column B
#                      18, #This tells the method to apply a column width of 18 pixels
#                      string_format #This applies the format 'string_template' to the column
#                     )

# writer.sheets['recommended_trades'].set_column('D:D', #This tells the method to apply the format to column B
#                      18, #This tells the method to apply a column width of 18 pixels
#                      string_format #This applies the format 'string_template' to the column
#                     )
# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [40]:
column_formats = {
    "A" : ["Ticker", string_format],
    "B" : ["Stock Price", dollar_format],
    "C" : ["Market Capitalization", dollar_format],
    "D" : ["Number of Shares to Buy", integer_format]
}

In [41]:
for column in column_formats.keys():
    writer.sheets["recommended_trades"].set_column(f"{column}:{column}",
                                                  22,
                                                  column_formats[column][1])
    
    writer.sheets["recommended_trades"].write(f"{column}1", column_formats[column][0], column_formats[column][1])


In [42]:
# writer.sheets["recommended_trades"].write("B1", "Stock Price", dollar_format)
# writer.sheets["recommended_trades"].write("C1", "Market Capitalization", dollar_format)
# writer.sheets["recommended_trades"].write("D1", "Number of Shares to Buy", integer_format)


## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()